In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Loading/Preprocessing data

def load_data(data_dir, img_size=(128, 128)):
    images, masks = [], []
    categories = ['Glioma', 'Meningioma', 'Pituitary tumor']
    
    for category in categories:
        folder = os.path.join(data_dir, category)
        for file in os.listdir(folder):
            if '_mask' not in file:
                img_path = os.path.join(folder, file)
                mask_path = os.path.join(folder, file.split('.')[0] + '_mask.png')
                
                # Load grayscale image and mask
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                
                if img is None or mask is None:
                    continue  # skip if file is broken
                
                # Resize
                img = cv2.resize(img, img_size)
                mask = cv2.resize(mask, img_size)
                
                # Preprocess
                img = cv2.GaussianBlur(img, (3, 3), 0)  # Blur to remove noise
                img = img / 255.0                       # Normalize Image
                mask = mask / 255.0                     # Normalize mask 
                
                images.append(img)
                masks.append(mask)
    
    return np.array(images), np.array(masks)

# Load data
X, Y = load_data('data/')

print(f"Loaded {len(X)} MRI scans.")


plt.figure(figsize=(10, 3))
for i in range(3):
    plt.subplot(2, 3, i+1)
    plt.imshow(X[i], cmap='gray')
    plt.title("MRI Image")
    plt.axis('off')
    
    plt.subplot(2, 3, i+4)
    plt.imshow(Y[i], cmap='Reds')
    plt.title("Ground Truth Mask")
    plt.axis('off')
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def simple_segmentation(img):
# Otsu's thresholding
    img_uint8 = (img * 255).astype(np.uint8) 
    _, thresh = cv2.threshold(img_uint8, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh


In [ ]:
def dice_coef(pred, truth):
    pred = pred.astype(np.float32).flatten()
    truth = truth.astype(np.float32).flatten()
    intersection = np.sum(pred * truth)
    return (2. * intersection + 1e-6) / (np.sum(pred) + np.sum(truth) + 1e-6)


In [ ]:
i = 0  # Choose a sample

original = X[i]
ground_truth = Y[i]
predicted = simple_segmentation(original)

# Dice score
dice = dice_coef(predicted, ground_truth)
print(f"Dice Score: {dice:.4f}")

# Show overlay
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(original, cmap='gray')
plt.title("Original MRI")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(predicted, cmap='Reds')
plt.title("Predicted Mask")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(original, cmap='gray')
plt.imshow(predicted, cmap='Reds', alpha=0.4)
plt.title("Overlay")
plt.axis('off')

plt.tight_layout()
plt.show()
